# 算法原理

在许多应用中，由于数据并不是线性可分的，因此PCA的优化目标并不能很好地适应这些数据。在这种情况下，核主成分分析（KPCA）算法被广泛使用。

核主成分分析（KPCA）是主成分分析（PCA）的一种扩展形式，它通过非线性映射将数据转换到高维空间中，然后在高维空间中进行PCA分析。KPCA可以在非线性数据上提取主成分，是一种有效的非线性降维方法。

KPCA的实现过程如下：

- 选择一个合适的核函数（如高斯核函数、多项式核函数等），将原始数据映射到高维空间中。

- 在高维空间中计算数据的协方差矩阵或者Gram矩阵，这个矩阵的维度通常很高。

- 对协方差矩阵或Gram矩阵进行特征值分解，得到每个特征向量和对应的特征值。

- 选取前k个特征向量，将原始数据映射到低维空间中。这些特征向量通常与原始数据的维度相同。

# 数据准备

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import mglearn
from sklearn.decomposition import KernelPCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score

In [24]:
#读取数据
data = pd.read_csv("../data/breast_cancer.csv")
print(data.head())

       0      1       2       3        4        5       6        7       8  \
0  17.99  10.38  122.80  1001.0  0.11840  0.27760  0.3001  0.14710  0.2419   
1  20.57  17.77  132.90  1326.0  0.08474  0.07864  0.0869  0.07017  0.1812   
2  19.69  21.25  130.00  1203.0  0.10960  0.15990  0.1974  0.12790  0.2069   
3  11.42  20.38   77.58   386.1  0.14250  0.28390  0.2414  0.10520  0.2597   
4  20.29  14.34  135.10  1297.0  0.10030  0.13280  0.1980  0.10430  0.1809   

         9  ...     21      22      23      24      25      26      27  \
0  0.07871  ...  17.33  184.60  2019.0  0.1622  0.6656  0.7119  0.2654   
1  0.05667  ...  23.41  158.80  1956.0  0.1238  0.1866  0.2416  0.1860   
2  0.05999  ...  25.53  152.50  1709.0  0.1444  0.4245  0.4504  0.2430   
3  0.09744  ...  26.50   98.87   567.7  0.2098  0.8663  0.6869  0.2575   
4  0.05883  ...  16.67  152.20  1575.0  0.1374  0.2050  0.4000  0.1625   

       28       29  label  
0  0.4601  0.11890      0  
1  0.2750  0.08902      0  
2 

In [25]:
#准备数据
data = data.dropna()
y = data['label']
x = data.drop(['label'],axis=1).astype('float64')

# 训练集、测试集划分
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state= 42)

# 模型训练

In [26]:
model = KernelPCA(n_components=10, kernel='rbf')
model.fit(x_train)

KernelPCA(kernel='rbf', n_components=10)

# 模型保存

In [27]:
# 法一
import joblib

# 保存模型
joblib.dump(model, '../test/best_models/kpca.pkl')

# 加载模型
model = joblib.load('../test/best_models/kpca.pkl')

In [28]:
# 法二
import pickle

with open('../test/best_models/kpca.pkl', 'wb') as f:
    pickle.dump(model, f)
 
#读取Model
with open('../test/best_models/kpca.pkl', 'rb') as f:
    model = pickle.load(f)

# 模型预测

In [29]:
# 转换数据
x_new = model.transform(x_train)

In [30]:
x_new.shape

(426, 10)

# 模型应用

In [31]:
# 先对数据降维，再进行分类
kpca = KernelPCA(n_components=10, kernel='rbf')
kpca.fit(x_train)
x_new = kpca.transform(x_train)

# 分类模型
knn = KNeighborsClassifier()
param_grid = {'n_neighbors': [2, 3, 4]}

#GridSearchCV优化参数、训练模型
gsearch = GridSearchCV(knn, param_grid)
knn = gsearch.fit(x_new, y_train)

#打印最优结果
print('KNN params:', knn.best_estimator_)

KNN params: KNeighborsClassifier(n_neighbors=3)


In [32]:
x_new = kpca.transform(x_test)
prediction = knn.predict(x_new)

In [33]:
# 计算准确率
acc = accuracy_score(y_test, prediction)
print("acc为：", acc)

acc为： 0.6223776223776224


In [34]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        54
           1       0.62      1.00      0.77        89

    accuracy                           0.62       143
   macro avg       0.31      0.50      0.38       143
weighted avg       0.39      0.62      0.48       143



D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
